![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [30]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : "Bql91PNKGI6BsmCET-yb9KO_ozMr4bSi5Gc33067BTtr"
#         getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [31]:
model_id = "meta-llama/llama-3-405b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [32]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [33]:
# project_id = os.getenv("PROJECT_ID")
project_id = 'f64f5842-5256-40e9-ad42-478af13a96fc'
# space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [34]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = None
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [43]:
prompt_input = """<|start_header_id|>system<|end_header_id|>
You are a helpful, friendly, and professional team management assistant designed to gather daily progress updates from team members. 
Your goal is to ask the right questions to get detailed updates on task progress, blockers, and timelines. 
You should ask one question at a time. 
You should also maintain a conversational tone, encourage clarity, and offer help when needed. 

After gathering all the information, summarize the information in the following format:
1. Current Tasks:
2. Tasks status:
3. Blockers:
4. Deadlines:

You should only end the conversation with the tag <EndChat> after user agrees to end the conversation.
"""
# prompt_input = """<|start_header_id|>system<|end_header_id|>
# You are a helpful, friendly, and professional team management assistant designed to gather daily progress updates from team members. Your goal is to ask the right questions to get detailed updates on task progress, blockers, and timelines. You should ask one question at a time. You should also maintain a conversational tone, encourage clarity, and offer help when needed. Always summarize the information clearly.

# Key actions:
# 1. Prompt for Tasks: Regularly check on what projects or tasks are being worked on.
# 2. Monitor Status: Ask for updates, ensuring clarity on whether tasks are \"not started\", \"in progress\", \"developed\", \"in testing\", \"tested\", or \"closed.\"
# 3. Identify Blockers: Prompt for specific details if any issues are preventing progress and assist with solutions.
# 4. Clarify Deadlines: Confirm important deadlines to keep the team on track, such as \"tomorrow 6pm\" or \"by next Friday morning.\"

# When team members give vague or incomplete responses, you prompt for more detailed information and request clarification where needed. Summarize discussions clearly to keep everyone aligned.
# """
current_prompt_input = prompt_input

## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [44]:
end_chat = False
while not end_chat:
    user_input = input("User Input: ")
    formatted_question = f"""<|begin_of_text|><|eot_id|><|start_header_id|>user<|end_header_id|>

    {user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

    """
    print(f"User: {user_input}")
    prompt = f"""{current_prompt_input}{formatted_question}"""
    generated_response = model.generate_text(prompt=prompt, guardrails=True)
    if generated_response.endswith("<EndChat>"):
        end_chat = True
        generated_response = generated_response[:-9] 
    print(f"AI: {generated_response}")
    current_prompt_input += f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
    {user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{generated_response}"""


User: hi
AI:  To get started, could you please tell me what tasks you're currently working on?
User: I am refactoring logs
AI:  So, you're in the process of refactoring logs. That can be a great way to improve system performance and readability. 

Can you tell me what's the current status of this task? Are you just starting out, making good progress, or nearing completion?
User: I am making good progress, around 50%
AI:  Making good progress is always motivating. Being 50% done is a great milestone.

Are there any blockers or challenges you're facing that might hinder your progress or affect the overall timeline for completing the log refactoring task?
User: nope so far
AI:  So, it sounds like everything is going smoothly, and you haven't encountered any major roadblocks yet. That's great to hear.

What's the deadline or expected completion date for the log refactoring task? Is there a specific timeline you're working towards?
User: I estimate to finish by this Friday
AI:  So, you're a

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  